In [1]:
from dotenv import load_dotenv
from tqdm import tqdm
from io import BytesIO
from zipfile import ZipFile
load_dotenv()
import os

from pyro_risks.utils.s3 import S3Bucket

s3 = S3Bucket(
	bucket_name="risk", 
	endpoint_url=os.environ["S3_ENDPOINT_URL"], 
	region_name=os.environ["S3_REGION"],
	aws_access_key_id=os.environ["S3_ACCESS_KEY"], 
	aws_secret_key=os.environ["S3_SECRET_KEY"]
)

zip_obj = s3.bucket.Object(key='datacube/datasets.tar.gz')

buffer = BytesIO(zip_obj.get()["Body"].read())

z = zipfile.ZipFile(buffer)
for filename in tqdm(z.namelist()):
    file_info = z.getinfo(filename)
    s3.s3.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=s3.bucket,
        Key=f'datacube/raw/{filename}'
    )

In [3]:
files = s3.list_files(pattern="datacube")